In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.ion()
# from dedalus.extras import plot_tools
# import pathlib
# from dedalus.tools import logging
# from dedalus.tools import post
# from dedalus.tools.parallel import Sync
nsnap = -1

def get_series(filename):
    file = h5py.File(filename, mode = 'r')
    vol_tr = file['tasks']['vol_tr']
    times = vol_tr.dims[0][0][:]
    return times, vol_tr[:].ravel()

def read_file(filedir,filename):
    
    file = h5py.File(filedir+filename, mode='r')
    u = file['tasks']['u']
    v = file['tasks']['v']
    p = file['tasks']['p']
    A = file['tasks']['A']
    B = file['tasks']['B']
    R = file['tasks']['R']
    S = file['tasks']['S']
    x = p[1][0]
    y = p[2][0]
    
    simtime = u.dims[0]['sim_time'][:]
    
    xgrid,ygrid = np.meshgrid(x,y, sparse = False, indexing = 'ij')
    

    # open the file in the write mode
    p = p[:,:,:]
    v = v[:,:,:]
    u = u[:,:,:]
    A = A[:,:,:]
    B = B[:,:,:]
    R = R[:,:,:]
    S = S[:,:,:]
    file.close()
    return p,v,u,A,B,R,S,xgrid,ygrid,simtime

def plot_all_basic_vars(u,v,p,A,B,R,S,xgrid,ygrid,simtime,plotdir,basename,ioffset,firstsnapshot,continuation):
    nsnapshots = np.shape(p)[0]
    iplot = ioffset
    figslist = os.listdir(plotdir)
    print(figslist)
    for nsnap in range(firstsnapshot,nsnapshots):
        fname = basename+str(iplot).zfill(4)+".png"
        makeplot = True
        if continuation:
            if fname in figslist:
                makeplot = False
            else:
                makeplot = True
                print(nsnap,simtime[nsnap],fname+" being created")
        if makeplot:
            

            fig, axs = plt.subplots(3, 2, sharex=True, sharey=True,figsize=(6,6))
            
            c00 = axs[0,0].pcolormesh(xgrid,ygrid,R[nsnap,:,:],cmap='inferno', shading = 'gouraud', 
                    vmin = R[nsnap,:,:].min(), vmax = R[nsnap,:,:].max() )
            fig.colorbar(c00,ax=axs[0,0])
            axs[0,0].set_xlabel('x')
            axs[0,0].set_ylabel('y')
            axs[0,0].set_title("r")

            c01 = axs[0,1].pcolormesh(xgrid,ygrid,S[nsnap,:,:],cmap='inferno', shading = 'gouraud', 
                    vmin = S[nsnap,:,:].min(), vmax = S[nsnap,:,:].max() )
            fig.colorbar(c01,ax=axs[0,1])
            # axs[0,1].set_xlabel('x')
            # axs[0,1].set_ylabel('y')
            axs[0,1].set_title("S")
            c10 = axs[1,0].pcolormesh(xgrid,ygrid,A[nsnap,:,:],cmap='inferno', shading = 'gouraud', 
                    vmin = A[nsnap,:,:].min(), vmax = A[nsnap,:,:].max() )
            fig.colorbar(c10,ax=axs[1,0])
            # axs[0,0].set_xlabel('x')
            # axs[0,0].set_ylabel('y')
            axs[1,0].set_title("A")

            c11 = axs[1,1].pcolormesh(xgrid,ygrid,B[nsnap,:,:],cmap='inferno', shading = 'gouraud', 
                    vmin = B[nsnap,:,:].min(), vmax = B[nsnap,:,:].max() )
            fig.colorbar(c11,ax=axs[1,1])
            # axs[0,1].set_xlabel('x')
            # axs[0,1].set_ylabel('y')
            axs[1,1].set_title("B")

            c20 = axs[2,0].pcolormesh(xgrid,ygrid,u[nsnap,:,:],cmap='inferno', shading = 'gouraud', 
                    vmin = u[nsnap,:,:].min(), vmax = u[nsnap,:,:].max() )
            fig.colorbar(c20,ax=axs[2,0])
            # axs[0,0].set_xlabel('x')
            # axs[0,0].set_ylabel('y')
            axs[2,0].set_title("u")

            c21 = axs[2,1].pcolormesh(xgrid,ygrid,v[nsnap,:,:],cmap='inferno', shading = 'gouraud', 
                    vmin = v[nsnap,:,:].min(), vmax = v[nsnap,:,:].max() )
            fig.colorbar(c21,ax=axs[2,1])
            # axs[0,1].set_xlabel('x')
            # axs[0,1].set_ylabel('y')
            axs[2,1].set_title("v")

            timestr = "$t=$" + "{0:4.2f}".format(simtime[nsnap])
            plt.figtext(0,0,timestr)
            plt.savefig(plotdir+fname,bbox_inches="tight")
            plt.close(fig)
        iplot +=1
    return iplot

In [2]:
import os

snapshotsdir = "./run00/"
plotdir = "./figs/"
basename = "all."
continuation = True
try:
    os.mkdir(plotdir)
except OSError:
    print("File directory already exists")
allfiles = os.listdir(snapshotsdir)
print(allfiles)
listfiles = []
for f in allfiles:
    if f.endswith(".h5"):
        listfiles.append(f)
listfiles.sort()
print(listfiles)
ioffset = 0
firstsnapshot = 0
for fname in listfiles:
    u,v,p,A,B,R,S,xgrid,ygrid,simtime = read_file(snapshotsdir,fname)
    print(fname)
    ioffet = plot_all_basic_vars(u,v,p,A,B,R,S,xgrid,ygrid,simtime,plotdir,basename,ioffset,firstsnapshot,continuation)
    
    print(ioffset)


['run00_s1', 'run00_s1.h5']
['run00_s1.h5']
run00_s1.h5
[]
0 0.0 all.0000.png being created
1 0.25000000000000017 all.0001.png being created
2 0.5000000000000003 all.0002.png being created
3 0.7500000000000006 all.0003.png being created
0
